In [1]:
def andm(x,y):
    x0,x1 = x[0],x[1]
    y0,y1 = y[0],y[1]
    z0 = ~y0 & (~y1 & x1 | y1 & x0) | y0 & (y1 & x1 | ~y1 & x0)
    return [z0,x1]

def andmn(x,y):
    x0,x1 = x[0],x[1]
    y0,y1 = y[0],y[1]
    z0 = ~y0 & (~y1 & ~x1 | y1 & ~x0) | y0 & (y1 & ~x1 | ~y1 & ~x0) % 2**64
    return [z0,x1]

In [2]:
import os
import random

def bool_split(x):
    r = urandom_to_int(size_bytes(x))
    xb = x ^ r
    res = [xb,r]
    return res

def bool_comb(x):
    return x[0] ^ x[1]

def arith_split(x):
    r = urandom_to_int(size_bytes(x))
    xa = x - r
    res = [xa,r]
    return res

def arith_comb(x):
    return (x[0] + x[1]) % 2**64

def b2a(x):
    xb,r = x[0],x[1]
    K = 64
    Km = 2**K
    y  = urandom_to_int(size_bytes(xb))
    T  = xb ^ y
    T  = (T  - y) % Km
    T  = T  ^ xb
    y  = y  ^ r
    xa = xb ^ y
    xa = (xa - y) % Km
    xa = xa ^ T
    res = [xa % Km,r % Km]
    return res
        
def a2b(x):
    xa,r = x[0],x[1]
    K = 64
    Km = 2**K
    y  = urandom_to_int(size_bytes(xa))
    T  = (2 * y) % Km
    xb = y ^ r
    O  = y & xb
    xb = T ^ xa
    y  = y ^ xb
    y  = y & r
    O  = O ^ y
    y  = T & xa
    O  = O ^ y
    for i in range(1,K):
        y = T & r
        y = y ^ O
        T = T & xa
        y = y ^ T
        T = (2 * y) % Km
    xb = xb ^ T
    res = [xb,r]
    return res

def b2a_s(xb,r):
    K = 64
    Km = 2**K
    y  = urandom_to_int(size_bytes(xb))
    T  = xb ^ y
    T  = T  - y % Km
    T  = T  ^ xb
    y  = y  ^ r
    xa = xb ^ y
    xa = xa - y % Km
    xa = xa ^ T
    return xa, r
        
def a2b_s(xa,r):
    K = 64
    Km = 2**K
    y  = urandom_to_int(size_bytes(xa))
    T  = 2 * y % Km
    xb = y ^ r
    O  = y & xb
    xb = T ^ xa
    y  = y ^ xb
    y  = y & r
    O  = O ^ y
    y  = T & xa
    O  = O ^ y
    for i in range(1,K):
        y = T & r
        y = y ^ O
        T = T & xa
        y = y ^ T
        T = 2 * y % Km
    xb = xb ^ T
    return xb, r

def urandom_to_int(byte_length):
    return random.randint(1, 2**64-1)
    # random_bytes = os.urandom(byte_length)
    # return int.from_bytes(random_bytes, 'big')

def size_bytes(number):
    bit_length = number.bit_length()
    byte_size = (bit_length + 7) // 8  # Round up to the nearest byte
    return byte_size

In [7]:
import binascii
import struct

initial_hash = (
    0x6a09e667f3bcc908,
    0xbb67ae8584caa73b,
    0x3c6ef372fe94f82b,
    0xa54ff53a5f1d36f1,
    0x510e527fade682d1,
    0x9b05688c2b3e6c1f,
    0x1f83d9abfb41bd6b,
    0x5be0cd19137e2179,
)

round_constants = (
    0x428a2f98d728ae22, 0x7137449123ef65cd, 0xb5c0fbcfec4d3b2f,
    0xe9b5dba58189dbbc, 0x3956c25bf348b538, 0x59f111f1b605d019,
    0x923f82a4af194f9b, 0xab1c5ed5da6d8118, 0xd807aa98a3030242,
    0x12835b0145706fbe, 0x243185be4ee4b28c, 0x550c7dc3d5ffb4e2,
    0x72be5d74f27b896f, 0x80deb1fe3b1696b1, 0x9bdc06a725c71235,
    0xc19bf174cf692694, 0xe49b69c19ef14ad2, 0xefbe4786384f25e3,
    0x0fc19dc68b8cd5b5, 0x240ca1cc77ac9c65, 0x2de92c6f592b0275,
    0x4a7484aa6ea6e483, 0x5cb0a9dcbd41fbd4, 0x76f988da831153b5,
    0x983e5152ee66dfab, 0xa831c66d2db43210, 0xb00327c898fb213f,
    0xbf597fc7beef0ee4, 0xc6e00bf33da88fc2, 0xd5a79147930aa725,
    0x06ca6351e003826f, 0x142929670a0e6e70, 0x27b70a8546d22ffc,
    0x2e1b21385c26c926, 0x4d2c6dfc5ac42aed, 0x53380d139d95b3df,
    0x650a73548baf63de, 0x766a0abb3c77b2a8, 0x81c2c92e47edaee6,
    0x92722c851482353b, 0xa2bfe8a14cf10364, 0xa81a664bbc423001,
    0xc24b8b70d0f89791, 0xc76c51a30654be30, 0xd192e819d6ef5218,
    0xd69906245565a910, 0xf40e35855771202a, 0x106aa07032bbd1b8,
    0x19a4c116b8d2d0c8, 0x1e376c085141ab53, 0x2748774cdf8eeb99,
    0x34b0bcb5e19b48a8, 0x391c0cb3c5c95a63, 0x4ed8aa4ae3418acb,
    0x5b9cca4f7763e373, 0x682e6ff3d6b2b8a3, 0x748f82ee5defb2fc,
    0x78a5636f43172f60, 0x84c87814a1f0ab72, 0x8cc702081a6439ec,
    0x90befffa23631e28, 0xa4506cebde82bde9, 0xbef9a3f7b2c67915,
    0xc67178f2e372532b, 0xca273eceea26619c, 0xd186b8c721c0c207,
    0xeada7dd6cde0eb1e, 0xf57d4f7fee6ed178, 0x06f067aa72176fba,
    0x0a637dc5a2c898a6, 0x113f9804bef90dae, 0x1b710b35131c471b,
    0x28db77f523047d84, 0x32caab7b40c72493, 0x3c9ebe0a15c9bebc,
    0x431d67c49c100d4c, 0x4cc5d4becb3e42b6, 0x597f299cfc657e2a,
    0x5fcb6fab3ad6faec, 0x6c44198c4a475817,
)


def _right_rotate(n: int, bits: int) -> int:
    return (n >> bits) | (n << (64 - bits)) & 0xFFFFFFFFFFFFFFFF


def sha512(message: str) -> str:
    if type(message) is not str:
        raise TypeError('Given message should be a string.')
    message_array = bytearray(message, encoding='utf-8')

    mdi = len(message_array) % 128
    padding_len = 119 - mdi if mdi < 112 else 247 - mdi
    ending = struct.pack('!Q', len(message_array) << 3)
    message_array.append(0x80)
    message_array.extend([0] * padding_len)
    message_array.extend(bytearray(ending))

    sha512_hash = list(initial_hash)
    for chunk_start in range(0, len(message_array), 128):
        chunk = message_array[chunk_start:chunk_start + 128]

        w = [0] * 80
        w[0:16] = struct.unpack('!16Q', chunk)

        for i in range(16, 80):
            s0 = (
                _right_rotate(w[i - 15], 1) ^
                _right_rotate(w[i - 15], 8) ^
                (w[i - 15] >> 7)
            )
            s1 = (
                _right_rotate(w[i - 2], 19) ^
                _right_rotate(w[i - 2], 61) ^
                (w[i - 2] >> 6)
            )
            w[i] = (w[i - 16] + s0 + w[i - 7] + s1) & 0xFFFFFFFFFFFFFFFF
        
        ar, br, cr, dr, er, fr, gr, hr = [0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]
        a, b, c, d, e, f, g, h = sha512_hash

        ar = bool_split(a)
        br = bool_split(b)
        cr = bool_split(c)
        dr = bool_split(d)
        er = bool_split(e)
        fr = bool_split(f)
        gr = bool_split(g)
        hr = bool_split(h) 

        arc = ar.copy()
        brc = br.copy()
        crc = cr.copy()
        drc = dr.copy()
        erc = er.copy()
        frc = fr.copy()
        grc = gr.copy()
        hrc = hr.copy()        
        
        sha512_hash = [arc,brc,crc,drc,erc,frc,grc,hrc]
        
        for i in range(80):
            # print(sha512_hash[0])
            sum0,sum1,ch,temp1,temp2,maj = [0,0],[0,0],[0,0],[0,0],[0,0],[0,0]
            
            sum1[0] = (
                _right_rotate(er[0], 14) ^
                _right_rotate(er[0], 18) ^
                _right_rotate(er[0], 41)
            )
            sum1[1] = (
                _right_rotate(er[1], 14) ^
                _right_rotate(er[1], 18) ^
                _right_rotate(er[1], 41)
            )
            
            ef = andm(er,fr)
            en = [0,0]
            eg = [0,0]
            e = bool_comb(er)
            en = bool_split(~e)
            eg = andm(en,gr)

            ch[0] = (ef[0]) ^ (eg[0])
            ch[1] = (ef[1]) ^ (eg[1])
            
            # b2a hr sum1 ch
            hr = b2a(hr)
            sum1 = b2a(sum1)
            ch = b2a(ch)
            ww = bool_split(w[i])
            ww = b2a(ww)
            rc = bool_split(round_constants[i])
            rc = b2a(rc)

            temp1[0] = (hr[0] + sum1[0] + ch[0] + rc[0] + ww[0]) % 2**64
            temp1[1] = (hr[1] + sum1[1] + ch[1] + rc[1] + ww[1]) % 2**64

            sum0[0] = (
                _right_rotate(ar[0], 28) ^
                _right_rotate(ar[0], 34) ^
                _right_rotate(ar[0], 39)
            )
            sum0[1] = (
                _right_rotate(ar[1], 28) ^
                _right_rotate(ar[1], 34) ^
                _right_rotate(ar[1], 39)
            )

            ab = andm(ar,br)
            ac = andm(ar,cr)
            bc = andm(br,cr)
            
            maj[0] = (ab[0]) ^ (ac[0]) ^ (bc[0])
            maj[1] = (ab[1]) ^ (ac[1]) ^ (bc[1])
            
            # b2a sum0 maj
            sum0 = b2a(sum0)
            maj = b2a(maj)
            
            temp2[0] = (sum0[0] + maj[0]) % 2**64
            temp2[1] = (sum0[1] + maj[1]) % 2**64
            
            hr[0],hr[1] = gr[0],gr[1]                                    #hr is bool
            gr[0],gr[1] = fr[0],fr[1]                                    #gr is bool
            fr[0],fr[1] = er[0],er[1]                                    #fr is bool
            
            #b2a dr
            dr = b2a(dr)
            x = dr[0]
            y = dr[1]
            q = temp1[0]
            v = temp1[1]
            
            er[0] = (x + q) % 2**64     #er is arith
            er[1] = (y + v) % 2**64     #er is arith
            dr[0],dr[1] = cr[0],cr[1]                                    #dr is bool
            cr[0],cr[1] = br[0],br[1]                                    #cr is bool
            br[0],br[1] = ar[0],ar[1]                                    #ar is bool
            ar[0] = (temp1[0] + temp2[0]) & 0xFFFFFFFFFFFFFFFF  #ar is arith
            ar[1] = (temp1[1] + temp2[1]) & 0xFFFFFFFFFFFFFFFF  #ar is arith
            
            ar = a2b(ar) 
            er = a2b(er)
            
        
        ar = b2a(ar) 
        er = b2a(er)
        hr = b2a(hr)
        gr = b2a(gr)
        fr = b2a(fr)
        dr = b2a(dr)
        cr = b2a(cr)
        br = b2a(br)
        
        sha512_hash[0]=b2a(sha512_hash[0])
        sha512_hash[1]=b2a(sha512_hash[1])
        sha512_hash[2]=b2a(sha512_hash[2])
        sha512_hash[3]=b2a(sha512_hash[3])
        sha512_hash[4]=b2a(sha512_hash[4])
        sha512_hash[5]=b2a(sha512_hash[5])
        sha512_hash[6]=b2a(sha512_hash[6])
        sha512_hash[7]=b2a(sha512_hash[7])  
        
        sha512_hash = [
            [(x[0] + y[0]) & 0xFFFFFFFFFFFFFFFF,(x[1] + y[1]) & 0xFFFFFFFFFFFFFFFF]
            for x, y in zip(sha512_hash, (ar, br, cr, dr, er, fr, gr, hr))
        ]
    
    sha512_hash_merge = []
    for i in range(len(sha512_hash)):
        sha512_hash_merge.append(arith_comb(sha512_hash[i]))

    return binascii.hexlify(
        b''.join(struct.pack('!Q', element) for element in sha512_hash_merge),
    ).decode('utf-8')

if __name__ == "__main__":
    assert sha512('Hello, world!') == (
        'c1527cd893c124773d811911970c8fe6e857d6df5dc9226bd8a160614c0cd963a4dde'
        'a2b94bb7d36021ef9d865d5cea294a82dd49a0bb269f51f6e7a57f79421'
    )
    # print(len(bin(int('0x'+sha512('Hello, world!')))))
    assert sha512('Python') == (
        'fd9d4d5b7a8a8fae6b1bc099b799110f7e4338606e2610f5d9506a4346e0c3bfbc525'
        'f4eed1e05aa8c6f46b8efff526ec48b500928a1b341ade5a7855f533932'
    )
    assert sha512('Illia') == (
        '09d4ba2426df452bddacde12436ef528c19e91236dfab2e7e2d45a0c2b450c7744397'
        '67d8c23ed5be08cd50a7aa74b4c7ffda135a2f9ac18b186893a6b052710'
    )
    assert sha512('ruahfiefiuwehifuewiufhweiufheiuhfieuwhfiuwefiruef') == (
        'ab806abd07432cb0dbba0afc1772597c63b869f215dc9a9b31185142fa2d859b87167'
        '976625d220f009b00a4c7795521da2988b9c50c6ac7819b18d1663ee53d'
    )
    
# print(sha512('
    
    
ef            = 1228922087630962688
eg            = 9223759666278706199
sum1          = 13859265365698444024
ch            = 10452681753909668887
temp1         = 5878021845030259271
sum0          = 14113587071880440340
maj_arith     = 18096561326811311912
temp2         = 13763404324982200636
a             = 1194682096302908291
b             = 15975067932647030380
c             = 7640891576956012808
d             = 13503953896175478587
e             = 10232707409967104626
f             = 4597624505582567432
g             = 5840696475078001361
h             = 11170449401992604703
    
    
sum1          = 13859265365698444024
ch            = 10452681753909668887
temp1         = 42771509992449362503
sum0          = 14113587071880440340
maj           = 18096561326811311912
temp2         = 32210148398691752252
a             = 1194682096302908291
b             = 15975067932647030380
c             = 7640891576956012808
d             = 13503953896175478587
e             = 10232707409967104626
f             = 4597624505582567432
g             = 5840696475078001361
h             = 11170449401992604703